In [1]:
import os
from google.colab import drive
drive.mount('/content/drive/')
os.chdir('/content/drive/My Drive/BPM')
root = os.getcwd()
!pwd

Mounted at /content/drive/
/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM


In [2]:
import gzip
import json
import pandas as pd
from tqdm import tqdm

# Download data

In [ ]:
os.chdir("./raw_data")
os.system("wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Electronics.json.gz")
os.system("gzip -d meta_Electronics.json.gz")

os.system("wget -c http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/meta_Office_Products.json.gz")
os.system("gzip -d meta_Office_Products.json.gz")

# Create df

In [ ]:
#Creating an empty list
data = []

#Decoding the gzip file
def parse(path):
  g = gzip.open(path, 'r')
  for l in g:
    yield json.dumps(eval(l))

#Defining f as the file that will contain json data
f = open("output_strict.json", 'w')

#Defining linebreak as '\n' and writing one at the end of each line
for l in parse(root + "/raw_data/meta_Electronics.json.gz"):
  f.write(l + '\n')
  
for l in parse(root + "/raw_data/meta_Office_Products.json.gz"):
  f.write(l + '\n')

#Appending each json element to the empty 'data' list
with open('output_strict.json', 'r') as f:
    for l in tqdm(f):
        data.append(json.loads(l))

#Reading 'data' as a pandas dataframe
full = pd.DataFrame(data)
full.to_csv(root+'/raw_data/raw.csv', index=False)
full.shape

633034it [00:32, 19713.55it/s]


(633034, 9)

In [ ]:
print(data[0])

{'asin': '0132793040', 'imUrl': 'http://ecx.images-amazon.com/images/I/31JIPhp%2BGIL.jpg', 'description': 'The Kelby Training DVD Mastering Blend Modes in Adobe Photoshop CS5 with Corey Barker is a useful tool for becoming familiar with the use of blend modes in Adobe Photoshop. For those who are serious about mastering all that Photoshop has to offer, mastering blend modes is just as important as mastering layers.In this DVD tutorial, seasoned expert Corey Barker explores the function of blend modes in a variety of scenarios such as image restoration, sharpening, adjustments, special effects and much more. Since every project scenario is different, Corey encourages you to experiment with these blend modes by giving you the skills and confidence you need.', 'categories': [['Electronics', 'Computers & Accessories', 'Cables & Accessories', 'Monitor Accessories']], 'title': 'Kelby Training DVD: Mastering Blend Modes in Adobe Photoshop CS5 By Corey Barker'}


In [ ]:
print(full.iloc[705])

asin                                                  9983901269
imUrl          http://ecx.images-amazon.com/images/I/31eSNcWV...
description                                                  NaN
categories     [[Electronics, Computers & Accessories, Cables...
title          Professional Kingston MicroSD 2GB (2 Gigabyte)...
price                                                        NaN
salesRank                 {'Cell Phones & Accessories': 2071844}
related                                                      NaN
brand                                                        NaN
Name: 705, dtype: object


# Delete rows with NaN

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv(root+"/raw_data/raw.csv")

# Drop unuseful fields
df = df.drop(['price', 'salesRank', 'related'], axis=1)

# Drop rows with NaN value in selected columns
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna(subset=['title', 'description', 'categories', 'imUrl', 'brand'])

df = df.reset_index(drop=True)

df.to_csv(root+"/cleaned_data/filtered.csv", index=False)

In [ ]:
df = pd.read_csv(root+'/cleaned_data/filtered.csv')
print(df.shape)
df.head()

(180002, 6)


,asin,imUrl,description,categories,title,brand
0,0594012015,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble
1,0594549507,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes &amp; Noble
2,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,Barnes &amp; Noble
3,0594033934,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes &amp;amp; Noble B505360439 Wren Quo...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B505360439 Wren Quote Cover,Barnes &amp; Noble
4,059400232X,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes &amp;amp; Noble B211-4023 Johnson Q...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B211-4023 Johnson Quote Cover,Barnes &amp; Noble


Check correctness

In [ ]:
nan_values = df[df['imUrl'].isna()]
print (nan_values)

Empty DataFrame
Columns: [asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['description'].isna()]
print (nan_values)

Empty DataFrame
Columns: [asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['title'].isna()]
print (nan_values)

Empty DataFrame
Columns: [asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['brand'].isna()]
print (nan_values)

Empty DataFrame
Columns: [asin, imUrl, description, categories, title, brand]
Index: []


In [ ]:
nan_values = df[df['description'] == '']
print (nan_values)

Empty DataFrame
Columns: [asin, imUrl, description, categories, title, brand]
Index: []


# Ensemble column

Creating data soup

In [ ]:
df['ensemble'] = df['title'] + ' ' + \
                 df['categories'].astype(str) + ' ' + \
                 df['brand']
                 
#print first record
df['ensemble'].iloc[100]

"Professional Kingston MicroSDHC 16GB (16 Gigabyte) Card for Samsung Galaxy Proclaim Phone with custom formatting and Standard SD Adapter. (SDHC Class 4 Certified) [['Electronics', 'Computers & Accessories', 'Cables & Accessories', 'Memory Cards', 'Micro SD Cards']] Kingston"

In [ ]:
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
forbidden_words = set(stopwords.words('english'))

#cleaning function
def text_cleaning(text):
  text = re.sub(r'amp','',text)
  text = re.sub(r'\s+',' ',re.sub('[^A-Za-z0-9]',' ', text.strip().lower())).strip()
  text = [word for word in text.split() if word not in forbidden_words]
  return ' '.join(text)

#apply text cleaning 
df['ensemble'] = df['ensemble'].apply(lambda text: text_cleaning(text))

df.to_csv(root+"/cleaned_data/preprocessed.csv")

#print example row
df['ensemble'].iloc[100]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyboardInterrupt: ignored

Range of ensemble field

In [ ]:
import pandas as pd
import nltk

# Calcola il numero di parole per ogni riga nella colonna 'text'
df['word_count'] = df['ensemble'].apply(lambda n: len(n.split()))

# Calcola il numero minimo e massimo di parole
min_words = df['word_count'].min()
max_words = df['word_count'].max()

# Stampa i risultati
print("Numero minimo di parole:", min_words)
print("Numero massimo di parole:", max_words)

In [ ]:
min_row_index = df['word_count'].idxmin()
print(df.loc[min_row_index])

In [ ]:
print(df.loc[5000])

In [ ]:
from requests import get
from io import BytesIO
from PIL import Image
import pandas as pd
import os

df = pd.read_csv(root+"/cleaned_data/preprocessed.csv")
d = df["imUrl"].iloc[5000]
print(d)
image_raw = get(df["imUrl"].iloc[0])
image = Image.open(BytesIO(image_raw.content))
width, height = image.size

print(width, height)

# Fine-tuning

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
import sys
!{sys.executable} -m pip install diffusers transformers accelerate scipy safetensors torch pandas

In [ ]:
import sys
print(sys.version)


3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]


##Dreambooth

In [ ]:
!pip install git
!git clone https://github.com/adobe-research/custom-diffusion.git
!cd custom-diffusion
!git clone https://github.com/CompVis/stable-diffusion.git
!cd stable-diffusion
!conda env create -f environment.yaml
!conda activate ldm
!pip install clip-retrieval tqdm

In [ ]:
!wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt

In [ ]:
## download dataset
# !wget https://www.cs.cmu.edu/~custom-diffusion/assets/data.zip
# !unzip data.zip

## run training (30 GB on 2 GPUs)
!bash scripts/finetune_real.sh "cat" data/cat real_reg/samples_cat  cat finetune_addtoken.yaml .

## save updated model weights
!python src/get_deltas.py --path logs/. --newtoken 1

## sample
!python sample.py --prompt "<new1> cat playing with a ball" --delta_ckpt logs/./checkpoints/delta_epoch\=000004.ckpt --ckpt .

bash: scripts/finetune_real.sh: No such file or directory
python3: can't open file '/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/src/get_deltas.py': [Errno 2] No such file or directory
python3: can't open file '/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/sample.py': [Errno 2] No such file or directory


In [ ]:
!bash scripts/finetune_gen.sh "cat" data/cat gen_reg/samples_cat  cat finetune_addtoken.yaml .

/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `bash scripts/finetune_gen.sh "cat" data/cat gen_reg/samples_cat  cat finetune_addtoken.yaml <pretrained-model-path>'


# Testing

In [ ]:
!git clone https://github.com/huggingface/diffusers

fatal: destination path 'diffusers' already exists and is not an empty directory.


In [ ]:
import os
root = "/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/"
os.chdir(root+'/diffusers')
!pwd

/content/drive/.shortcut-targets-by-id/1rGkjnEE_3Wi6ptkR36DxTvGqAlmsVmg0/BPM/diffusers


In [ ]:
!pip install -e .

In [ ]:
!pip install accelerate torchvision transformers>=4.25.1 ftfy tensorboard Jinja2 clip-retrieval  

In [ ]:
!accelerate config default

2023-05-23 14:12:52.452463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!pip install clip-retrieval
!python retrieve.py --class_prompt cat --class_data_dir real_reg/samples_cat --num_class_images 200

In [ ]:
!export MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export OUTPUT_DIR="."
!export INSTANCE_DIR="./data/cat"
!accelerate launch train_custom_diffusion.py \
  --pretrained_model_name_or_path=$MODEL_NAME  \
  --instance_data_dir=$INSTANCE_DIR \
  --output_dir=$OUTPUT_DIR \
  --class_data_dir=./real_reg/samples_cat/ \
  --with_prior_preservation --real_prior --prior_loss_weight=1.0 \
  --class_prompt="cat" --num_class_images=200 \
  --instance_prompt="photo of a <new1> cat"  \
  --resolution=512  \
  --train_batch_size=2  \
  --learning_rate=1e-5  \
  --lr_warmup_steps=0 \
  --max_train_steps=250 \
  --scale_lr --hflip  \
  --modifier_token "<new1>" \
  --validation_prompt="<new1> cat sitting in a bucket" 

# Descrizioni

In [3]:
df = pd.read_csv(root+'/cleaned_data/filtered.csv')
df.head()

,asin,imUrl,description,categories,title,brand
0,0594012015,http://ecx.images-amazon.com/images/I/41kQsHT8...,The Barnes &amp;amp; Noble BNADPN31 Power Kit ...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble NOOK Power Kit in Carbon BN...,Barnes &amp; Noble
1,0594549507,http://ecx.images-amazon.com/images/I/01eX9rjx...,"Slim, smart, and lightweight, this ingenious c...","[['Electronics', 'eBook Readers & Accessories'...",Groovy Protective Stand Cover: Custom Designed...,Barnes &amp; Noble
2,0594481813,http://ecx.images-amazon.com/images/I/41K7ymN5...,Power up your device with this Barnes &amp; No...,"[['Electronics', 'eBook Readers & Accessories'...",Barnes &amp; Noble OV/HB-ADP Universal Power Kit,Barnes &amp; Noble
3,0594033934,http://ecx.images-amazon.com/images/I/31Vl-YJ5...,The Barnes &amp;amp; Noble B505360439 Wren Quo...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B505360439 Wren Quote Cover,Barnes &amp; Noble
4,059400232X,http://ecx.images-amazon.com/images/I/31zdIqAs...,The Barnes &amp;amp; Noble B211-4023 Johnson Q...,"[['Electronics', 'Computers & Accessories', 'P...",Barnes &amp; Noble B211-4023 Johnson Quote Cover,Barnes &amp; Noble


In [5]:
import requests
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()

def check_url(url):
  response = requests.get(url)
  if response.status_code == 200:
      return True
  else:
      return False

df['valid'] = df['imUrl'].progress_apply(lambda url: check_url(url))

<ipython-input-5-3a1ec97c1d43>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

  0%|          | 0/180002 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
non_valid = df[df['valid'] == False]
print (non_valid)

In [ ]:
import re
import nltk


#cleaning function
def text_cleaning(text):
  text = bytes(text, "utf-8").decode("unicode_escape")
  text = text.strip()
  return text

#apply text cleaning 
df['description'] = df['description'].apply(lambda text: text_cleaning(text))

# df.to_csv(root+"/cleaned_data/preprocessed.csv")

#print example row
df['description'].iloc[100]

In [ ]:
cleaned = df[df['description']]
print (cleaned)

In [ ]:
#apply text cleaning 
df['title'] = df['title'].apply(lambda text: text_cleaning(text))

#print example row
df['title'].iloc[100]

In [ ]:
#apply text cleaning 
df['brand'] = df['brand'].apply(lambda text: text_cleaning(text))

#print example row
df['brand'].iloc[100]

In [ ]:
df['categories'] = df['categories'].replaceAll("[]'", "");

In [ ]:
df['sub_category'] = df['categories'].split(',')[-1]